## Imports

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss, accuracy_score, recall_score, make_scorer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

## Reading Data

In [2]:
# reading CSVs
vehicle_df = pd.read_csv('../data/localdata/Traffic_Crashes_Vehicles.csv')
people_df = pd.read_csv('../data/localdata/Traffic_Crashes_People.csv')
crash_df = pd.read_csv('../data/localdata/Traffic_Crashes_Crashes.csv')

/Users/robertdaly/Downloads/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/robertdaly/Downloads/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (20,24,25,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Data Cleaning

In [3]:
# Crashes DataFrame

crash_df.drop(columns=['WORK_ZONE_I', 'WORK_ZONE_TYPE', 'DOORING_I', 'WORKERS_PRESENT_I',
                                          'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'RD_NO', 'REPORT_TYPE', 'CRASH_DATE_EST_I'], axis=1, inplace=True)
crash_df.dropna(subset = ['INJURIES_TOTAL', 'LATITUDE', 'MOST_SEVERE_INJURY', 
                          'STREET_DIRECTION', 'BEAT_OF_OCCURRENCE'], axis=0, inplace=True)
crash_df.columns = crash_df.columns.str.title()
crash_df.columns = crash_df.columns.str.replace('_', ' ')

In [4]:
# Vehicles DataFrame

vehicle_df = vehicle_df[['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'CRASH_DATE', 'UNIT_NO', 'UNIT_TYPE',
                       'VEHICLE_YEAR', 'VEHICLE_USE', 'VEHICLE_TYPE', 'VEHICLE_DEFECT', 'MANEUVER', 'OCCUPANT_CNT',
                         'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I',
                       'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I', 'AREA_10_I', 'AREA_11_I',
                       'AREA_12_I', 'AREA_99_I', 'FIRST_CONTACT_POINT']]
vehicle_df.dropna(subset = ['VEHICLE_USE', 'FIRST_CONTACT_POINT', 'UNIT_TYPE'], axis=0, inplace=True)
vehicle_df['VEHICLE_YEAR'].fillna(value = 'Unknown', inplace=True)

vehicle_df.columns = vehicle_df.columns.str.title()
vehicle_df.columns = vehicle_df.columns.str.replace('_', ' ')

In [5]:
# People DataFrame

people_df.drop(columns=['RD_NO', 'CELL_PHONE_USE', 'PEDPEDAL_ACTION', 
                        'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'SEAT_NO', 
                        'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'BAC_RESULT', 'BAC_RESULT VALUE', 
                        'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'CITY', 'STATE', 'ZIPCODE']
               , axis=1, inplace=True)

people_df.dropna(subset = ['AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'VEHICLE_ID', 
                           'SAFETY_EQUIPMENT', 'SEX'], axis=0, inplace=True)

people_df.columns = people_df.columns.str.title()
people_df.columns = people_df.columns.str.replace('_', ' ')

## Feature Engineering Crash Score

In [6]:
# Injury Score

crash_df[['Injuries Total', 'Injuries Fatal', 'Injuries Incapacitating', 
            'Injuries Non Incapacitating', 'Injuries Reported Not Evident']][crash_df['Injuries Total']!=0][:50]

crash_df['total injured'] = crash_df['Injuries Fatal'] + crash_df['Injuries Incapacitating'] + crash_df['Injuries Non Incapacitating'] + crash_df['Injuries Reported Not Evident']

# Checking if added up columns in 'total injured' make up 'Injuries Total'
(crash_df['total injured'] == crash_df['Injuries Total']).value_counts()

# Injury Score column is a linear combination of the factors that make up Injuries Total simply multiplied by a constant depending on their severity
crash_df['Injury Score'] = crash_df['Injuries Fatal']*7 + crash_df['Injuries Incapacitating']*3 + crash_df['Injuries Non Incapacitating']*2 + crash_df['Injuries Reported Not Evident']

# Accounting Material Damage of Car and Surrounding Areas )
# Crash Score 

crash_df['Damage'].value_counts()
mapping = {'OVER $1,500': 3, '$501 - $1,500': 2, "$500 OR LESS": 1}
crash_df['Damage_ODE'] = crash_df['Damage'].map(mapping)
crash_df['Injury Score'] = crash_df['Injury Score'] + crash_df['Damage_ODE']*3
crash_df['Crash Score'] = crash_df['Injury Score']

## Injury Classification Column (Predictor)

In [7]:
# def fill_injury_class_rows(row):
#     if row["Injuries Fatal"] > 0 :
#         return 'Fatal'
#     elif row["Injuries Incapacitating"] > 0 :
#         return 'Incapacitating'
#     elif row['Injuries Non Incapacitating'] + row['Injuries Reported Not Evident'] > 0:
#         return 'Minor'
#     else:
#         return 'None Injured'

# crash_df['Injury Classification'] = crash_df.apply(fill_injury_class_rows, axis=1)

# crash_df['Injury Classification']

In [8]:
def fill_fatal_class_rows(row):
    if row["Injuries Fatal"] > 0 :
        return 'Fatal'
    else:
        return 'Not Fatal'

crash_df['Fatality Classification'] = crash_df.apply(fill_fatal_class_rows, axis=1)

In [9]:
crash_df['Fatality Classification'].value_counts(normalize=True)


Not Fatal    0.998911
Fatal        0.001089
Name: Fatality Classification, dtype: float64

In [10]:
crash_df['First Crash Type'].value_counts()

PARKED MOTOR VEHICLE            174158
REAR END                        169065
SIDESWIPE SAME DIRECTION        113196
TURNING                         106391
ANGLE                            81087
FIXED OBJECT                     34661
PEDESTRIAN                       17168
PEDALCYCLIST                     11217
SIDESWIPE OPPOSITE DIRECTION     10633
OTHER OBJECT                      7189
REAR TO FRONT                     6784
HEAD ON                           6387
REAR TO SIDE                      4040
OTHER NONCOLLISION                2288
REAR TO REAR                      1437
ANIMAL                             529
OVERTURNED                         446
TRAIN                               39
Name: First Crash Type, dtype: int64

In [11]:
crash_df['Prim Contributory Cause'].value_counts()

UNABLE TO DETERMINE                                                                 288754
FAILING TO YIELD RIGHT-OF-WAY                                                        81802
FOLLOWING TOO CLOSELY                                                                73712
NOT APPLICABLE                                                                       39220
IMPROPER OVERTAKING/PASSING                                                          36676
FAILING TO REDUCE SPEED TO AVOID CRASH                                               31699
IMPROPER BACKING                                                                     29960
IMPROPER LANE USAGE                                                                  27000
IMPROPER TURNING/NO SIGNAL                                                           24786
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  24684
DISREGARDING TRAFFIC SIGNALS                                                         14530

## Merging Vehicle and Crash DataFrames

In [12]:
vehicles_crashes_df = vehicle_df.merge(crash_df, on = 'Crash Record Id', how = 'inner')
vehicles_crashes_df.drop_duplicates(subset='Crash Record Id', inplace=True)

In [13]:
categorical = ['Weather Condition', 'Roadway Surface Cond', 'Road Defect', 'Alignment',
                       'Traffic Control Device', 'Device Condition', 'Crash Hour', 'Trafficway Type', 'Maneuver', 'Vehicle Defect',
               'Lighting Condition', 'First Crash Type', 'Prim Contributory Cause', 'Sec Contributory Cause'] #potentiall vehicle defects

ohe = OneHotEncoder(drop='first')
ohe.fit(vehicles_crashes_df[categorical])

categorical_encoded = pd.DataFrame(ohe.transform(vehicles_crashes_df[categorical]).todense(),
                               columns=ohe.get_feature_names())
# for Nick's version of OneHotEncoder, since its newer, the attribute is get_feature_names_out but get_feature_name for bobby and mike's version

In [14]:
categorical_encoded.shape

(745455, 238)

In [15]:
vehicles_crashes_df.shape

(745455, 70)

## Creating Train_Test_Split

In [16]:
#X = categorical_encoded
#y = vehicles_crashes_df['Fatal Classification']

#X_train, X_test, y_train, y_test = train_test_split(X, y)

## Creating Dummy Model

In [17]:
#Instantiate dummy model that will always predict majority class
#dummy_model = DummyClassifier(strategy="most_frequent", random_state = 42)
#dummy_model.fit(X_train, y_train)

#baseline score
#dummy_model.score(X_train, y_train)

In [18]:
#y_train.value_counts(normalize=True)

In [19]:
#can also grab cross_val mean to see how it’s performing
#cv_results = cross_val_score(dummy_model, X_train, y_train, cv=5)
#cv_results.mean()

## Testing Dummy Model

In [20]:
#dummy_model.score(X_test, y_test)

## Creating Logistic Regression Model

In [21]:
# logreg = LogisticRegression(random_state=42, max_iter = 1000)
# log_model = logreg.fit(X_train, y_train)

# y_hat_train = log_model.predict(X_train)
# y_hat_test = log_model.predict(X_test)

In [22]:
#plot_confusion_matrix(log_model, X_train, y_train);

In [23]:
# print("Training Score: ", log_model.score(X_train, y_train))
# print("Testing Score: ", log_model.score(X_test, y_test))

## SMOTE

In [24]:
#CAUTION! This code takes about 20 minutes to run.
# # Instantiate our SMOTE
# sm = SMOTE(random_state=42)
# # Fit and resample on the training data
# X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

In [25]:
#CAUTION! This code takes about 2 hours to run. 

# #Run Log Model on resampled SMOTE data
# log_model = logreg.fit(X_train_smote, y_train_smote)

# print("Training Score: ", log_model.score(X_train_smote, y_train_smote))
# print("Testing Score: ", log_model.score(X_test, y_test))

In [26]:
#y_train_smote.value_counts()

In [27]:
#plot_confusion_matrix(log_model, X_train_smote, y_train_smote);

Looking at this confusion matrix, it appeared that the model was performing well when predicting Fatal and None Injured. As Vision Zero's goal is for zero fatalities, we decided to hone our focus on Fatal vs. Not Fatal.

## Decision Tree Model

In [28]:
# normal decision tree model
# clf = DecisionTreeClassifier(criterion='gini', random_state=42, )

# clf.fit(X_train, y_train)

# y_preds = clf.predict(X_test)

# print('Accuracy: ', accuracy_score(y_test, y_preds))

# smote decision tree model
# clf = DecisionTreeClassifier(criterion='gini', random_state=42, max_depth =10)

# clf.fit(X_train_smote, y_train_smote)

# y_preds = clf.predict(X_test)

# print('Accuracy: ', accuracy_score(y_test, y_preds))

## Creating Train_Test_Split for Fatal vs. Not Fatal

In [29]:
vehicles_crashes_df['Fatality Classification'].value_counts(normalize=True)

Not Fatal    0.998913
Fatal        0.001087
Name: Fatality Classification, dtype: float64

In [30]:
X = categorical_encoded
y = vehicles_crashes_df['Fatality Classification']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Creating Dummy Model

In [31]:
#Instantiate dummy model that will always predict majority class
dummy_model = DummyClassifier(strategy="most_frequent", random_state = 42)
dummy_model.fit(X_train, y_train)

y_true = y_train
y_pred = dummy_model.predict(X_train)
#baseline score
recall_score(y_true, y_pred, pos_label='Fatal')

0.0

#### Ensuring we are evaluating models based on recall.
False Negatives in this case means that our model predicted a crash to be Not Fatal when in actuality it was Fatal. As lives are of the upmost importance, we want to minimize that chance of a False Negative as much as possible. 

In [32]:
recall_scorer = make_scorer(recall_score, pos_label='Fatal')

In [33]:
cv_score_dummy = cross_val_score(estimator = dummy_model, X=X_train, y=y_train, scoring = recall_scorer, n_jobs=-1).mean()
cv_score_dummy

0.0

## Creating a Logistic Regression Model

In [34]:
logreg = LogisticRegression(random_state=42, max_iter = 1000, class_weight = 'balanced')
log_model = logreg.fit(X_train, y_train)
y_true = y_train
y_pred = log_model.predict(X_train)
recall_score(y_true, y_pred, pos_label = 'Fatal')

KeyboardInterrupt: 

In [ ]:
logreg = LogisticRegression(random_state=42, max_iter = 2000, class_weight = 'balanced', penalty = 'none')
log_model = logreg.fit(X_train, y_train)
y_true = y_train
y_pred = log_model.predict(X_train)
recall_score(y_true, y_pred, pos_label = 'Fatal')

In [ ]:
Attempting to flip our confusion matrix so Fatal is positive label
cm = confusion_matrix(y_true, y_pred).T
cm

In [ ]:
plot_confusion_matrix(log_model, X_train, y_train, labels = ['Not Fatal', 'Fatal']);

In [ ]:
y_train

In [ ]:
log_model.classes_

In [ ]:
cv_score_log = cross_val_score(estimator = log_model, X=X_train, y=y_train, scoring = recall_scorer, n_jobs=4).mean()
cv_score_log

#### Logistic Regression Pipeline including SMOTE and Model

In [ ]:
#enter non-smoted data into pipeline

In [39]:
logpipe = Pipeline([('smote', SMOTE(random_state=42, sampling_strategy = 0.3)),
                    ('model', logreg)])

In [ ]:
#Cross Validating LogReg with Pipeline
cv_score_logpipe = cross_val_score(estimator=logpipe, X=X_train, y=y_train, scoring = recall_scorer).mean()
cv_score_logpipe

#### Most likely will delete as SMOTE is now done in pipeline ---- Using SMOTE to account for our class imbalance

In [36]:
#Most likely will delete as SMOTE is done in pipeline now
sm = SMOTE(random_state=42, sampling_strategy = 0.3)

X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)



In [37]:
#Most likely will delete as SMOTE is done in pipeline now
log_model_smote = logreg.fit(X_train_smote, y_train_smote)

y_true_smote = y_train_smote
y_pred_smote = log_model_smote.predict(X_train_smote)

print("Training recall: ", recall_score(y_true_smote, y_pred_smote, pos_label = 'Fatal'))

KeyboardInterrupt: 

In [ ]:
#Most likely will delete as SMOTE is done in pipeline now
cv_scores_smote = cross_val_score(estimator = log_model_smote, X=X_train_smote, y=y_train_smote, 
                                  scoring = recall_scorer, n_jobs = -1).mean()
cv_scores_smote

In [ ]:
#Most likely will delete as SMOTE is done in pipeline now
log_model_smote.coef_

## Decision Tree Model

In [ ]:
#normal decision tree model
clf = DecisionTreeClassifier(criterion='gini', random_state=42, class_weight = 'balanced', max_depth = 24, min_samples_split = 5000)

clf.fit(X_train, y_train)

y_preds = clf.predict(X_train)

print('Recall: ', recall_score(y_train, y_preds, pos_label = 'Fatal'))

In [ ]:
#Cross Validating clf
cv_score_clf = cross_val_score(estimator = clf, X=X_train, y=y_train, scoring = recall_scorer, n_jobs=-1).mean()
cv_score_clf

#### Decision Tree Pipeline for clf with SMOTE and Model

In [ ]:
#Create Pipeline
clfpipe = Pipeline([('smote', SMOTE(random_state=42, sampling_strategy = 0.3)),
                    ('model', clf)])

In [ ]:
#Cross Validating Decision Tree with pipeline
cv_score_clfpipe = cross_val_score(estimator=clfpipe, X=X_train, y=y_train, scoring = recall_scorer).mean()
cv_score_clfpipe

## Decision Tree Revised 1 (clf_1)

In [ ]:
#Instantiating Decision Tree with new parameters to increase recall score
clf_1 = DecisionTreeClassifier(criterion='gini', random_state=42, class_weight = 'balanced', max_depth = 24,
                                   min_samples_split = 2500)

clf_1.fit(X_train, y_train)

y_preds = clf_1.predict(X_train)

print('Recall: ', recall_score(y_train, y_preds, pos_label = 'Fatal'))

#### Decision Tree Pipeline for clf_1 with SMOTE and Model

In [ ]:
#Create Pipeline with new model
clf_1pipe = Pipeline([('smote', SMOTE(random_state=42, sampling_strategy = 0.3)),
                    ('model', clf_1)])

In [ ]:
#Cross Validating Decision Tree with pipeline
cv_score_clf_1pipe = cross_val_score(estimator=clf_1pipe, X=X_train, y=y_train, scoring = recall_scorer).mean()
cv_score_clf_1pipe

### Organizing Feature Importance Data for further analysis

In [ ]:
feature_importance = clf_smote.feature_importances_
feature_importance

In [ ]:
categorical_encoded.columns

In [ ]:
feature_importances = pd.Series(list(feature_importance), name = 'feature_importance')
features = pd.Series(list(categorical_encoded), name = 'features')


In [ ]:
features

In [ ]:
feature_importances

In [ ]:
features_df = pd.DataFrame([feature_importances, features]).transpose()

In [ ]:
features_df

In [ ]:
features_df.sort_values(by='feature_importance', inplace=True, ascending=False)

In [ ]:
features_df.head(15)

## Feature Importance Analysis

In [ ]:
#Check to see if indices align
X.head()

In [ ]:
#Check to see if indices align
y.head()

In [ ]:
#reset y index so can merge with X on the index
y.reset_index(drop = True, inplace = True)

In [ ]:
#Confirm reset was successful
y.head()

In [ ]:
#Merge dfs to be able to perform calculations with all data together
categorical_and_fatal_df = X.merge(y, left_index = True, right_index = True, how = 'right')

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
#Confirm merge was successful. Should have same # of rows as before with one more column than X
categorical_and_fatal_df.shape

In [ ]:
#Function takes in a dataframe and relevant columns.
#Function loops through each column, calculating the percentage of Fatalities for each column's involvment in crashes
#Function outputs a dataframe with column names and their Fatality Percentage in descending order
def calc_percentage(df, columns):
    
    percent_dict = {}
    
    for col in columns:    
        #Calculate percentage
        col_perc = len(df.loc[(df[col]==1) & (df['Fatality Classification']=='Fatal')]) / len(df.loc[df[col]==1])
        #Add percentage to dictionary
        percent_dict[col] = col_perc
    #Convert dictionary to DataFrame
    percent_df = pd.DataFrame.from_dict(percent_dict, orient='index', columns=['Fatality Percentage'])
    percent_df.sort_values(by = 'Fatality Percentage', ascending = False, inplace = True)
    
    return percent_df

In [ ]:
relevant_columns = features_df['features'].unique()[:15].tolist() #select columns to analyze the fatal percentage of
feature_fatal_percent_df = calc_percentage(categorical_and_fatal_df, relevant_columns)
feature_fatal_percent_df

In [ ]:
#x6 is Crash Hour, 1 corresponding to 1pm and 21 corresponding to 9pm
vehicles_crashes_df['Crash Hour'].value_counts()

In [ ]:
#Group by index for easier group analysis
grouped_feature_fatal_percent_df = feature_fatal_percent_df.sort_index()
grouped_feature_fatal_percent_df

In [ ]:
#Output df to csv for graph making in Tableau
feature_fatal_percent_df.to_csv('../data/localdata/feature_fatal_percent.csv', index=False)
grouped_feature_fatal_percent_df.to_csv('../data/localdata/grouped_feature_fatal_percent.csv', index=False)

## Testing Data

In [ ]:
y_preds_test = clf_smote.predict(X_test)
print('Recall: ', recall_score(y_test, y_preds_test, pos_label = 'Fatal'))

In [ ]:
y_pred_test = log_model_smote.predict(X_test)

print("Training recall: ", recall_score(y_test, y_pred_test, pos_label = 'Fatal'))